In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
from itertools import cycle
import seaborn as sns
import statsmodels.api as sm 
from scipy.interpolate import interp1d
import joblib

pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

# 1 データ
## データの詳細
### sales_train_validationについて
#### 概要

製品およびストアごとの過去の1日の販売台数データが含まれている。  
主トレーニングデータとなっており、 2011-01-29から2016-04-24までの1913日ごとに1つの列がある。  
アイテム、部門、カテゴリ、ストア、および状態のIDも含まれている。  
3049アイテムと10店舗のすべての組み合わせの行数は30490。

* id: id
* item_id: アイテム(商品)のid
* dept_id: 部門のid
* cat_id: カテゴリーのid
* store_id: 店舗のid
* state_id: 州のid
* d_hoge: 日にちごとの販売数(売り上げの合計ではない)

### sell_prices.csvについて
#### 概要

店舗ごとに販売された製品の価格と日付に関する情報。  
ストアIDとアイテムID、およびアイテム販売価格の週平均。  
この価格表にはトレーニングデータと検証データにリンクするためのstore_idとitem_idが記述されている。  
価格は$ 0.10から100ドルを少し超える範囲。

* store_id: 店舗のid
* item_id: アイテム(商品)のid
* wm_yr_wk: 週のid
* sell_price: 販売価格


### calendar.csvについて
#### 概要

製品が販売された日付に関する情報。  
2011-01-29から2016-05-30までの1949日分のカレンダーのデータ。  
日付、曜日、月、年と関連イベント、および三つの州がこの日にSNAPフードスタンプ（訳者注：補助的栄養支援プログラム、無料フードクーポン）での購入を許可したかを示すバイナリフラグ（1:許可、0:不許可）。  
event_name_2列には、NA以外の行が5行しかない。  
特定の日に複数のイベントが発生するのは5回のみです。

* date: 日付
* wm_yr_wk: 週のid
* weekday: 曜日
* wday: 曜日のid(土曜日が1, 金曜日が7)
* month: 月
* year: 年
* d: 日
* event_name_1: イベントの名前
* event_type_1: イベントの詳細(スポーツなど)
* event_name_2: イベントの名前
* event_type_2: イベントの詳細(スポーツなど)
* snap_CA: SNAPフードスタンプのフラグ(カリフォルニア州)
* snap_TX: SNAPフードスタンプのフラグ(テキサス州)
* snap_WI: SNAPフードスタンプのフラグ(ウィスコンシン州)

### sample_submission.csvについて
#### 概要

提出の正しい形式。  
詳細については、[評価]タブを参照。

* id: id
* F1 - F28: 日付の売上数(28日分を予測)


In [ ]:
# 各種データのインプット
osj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
sales = pd.read_csv(osj(INPUT_DIR, 'sales_train_evaluation.csv'))
prices = pd.read_csv(osj(INPUT_DIR, 'sell_prices.csv'))
calendar = pd.read_csv(osj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(osj(INPUT_DIR, 'sample_submission.csv'))

In [ ]:
# salesデータの確認
sales.head()

In [ ]:
# 販売価格データの確認
prices.head()

In [ ]:
# カレンダーデータの確認
calendar['date'] = pd.to_datetime(calendar['date']) # dateをdatetime型に変換
calendar.head()

In [ ]:
# 提出ファイル例の確認
sample_submit.head()

# 2 データ分析

ランダムに選出した6個の商品の売上数をプロット。  
プロット結果から以下のことが考えられる。

* 毎日売れているものは少なそう
* 途中から販売が開始しているようなもの(途中から売上数が0ではなくなる)が存在している
* 正確な予測のために多くのノイズに対処する必要がある

In [ ]:
# ランダムに抽出（個数、random_stateは任意）
examples = sales.sample(6, random_state=700)
# calendarとマージ
d_cols = [c for c in sales.columns if 'd_' in c] # d_で始まる日付列のリスト
examples = examples.set_index('id')[d_cols].T.merge(calendar.set_index('d')['date'],left_index=True, 
                                                    right_index=True, validate='1:1').set_index('date')

In [ ]:
# 個数のプロット(縦軸: 売り上げ個数 横軸: 日付)
for item in examples.columns:
    examples[item].plot(title=item,
                        figsize=(15, 2),
                        color=next(color_cycle))
    plt.show()


# 2.1 全てのデータについて可視化
全てのアイテム、店舗、カテゴリ、部門、および売上の集計時系列をプロットする。  
プロット結果から以下のことが考えられる。  

* 全体を通してみると、売り上げは徐々に増加していることがわかる
* 12/25で毎年売り上げが極端に少なくなっている
* 1年において、売り上げが高い時期とそうでない時期が周期的に訪れている

In [ ]:
# 過去の売り上げのマージ
past_sales = sales.set_index('id')[d_cols].T.merge(calendar.set_index('d')['date'],
                                                   left_index=True,
                                                   right_index=True,
                                                   validate='1:1').set_index('date')

In [ ]:
# プロット
past_sales.sum(axis=1).plot(
    figsize=(15, 5),
    alpha=0.8,
    title='Total Sales')
plt.show()

In [ ]:
# 極端に少ない日の確認
[past_sales.index[_] for _, i in enumerate(past_sales.sum(axis=1)) if i <= 10000]

# 2.2 月毎のデータについて可視化
1年における売り上げの推移を詳しくみるために月毎(正確には30日ごと)に平均をとった売り上げのプロットを行う。  
また、このプロットは州ごとに推移を出すこととする。  
プロット結果から以下が考えられる。

* CAは高い売上数を維持している
* WIは最後の方でTXを抜いている


In [ ]:
# 州のリストの作成
state_list = sales['state_id'].unique()

# プロット
for i in state_list:
    items_col = [c for c in past_sales.columns if i in c]
    # 各アイテムについて30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[items_col].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        alpha=0.8,
        title='Rolling 30 Day Average Total Sales by State')
plt.legend(sales['state_id'].unique())
plt.show()


# 2.3 カテゴリーごとの可視化
カテゴリーは以下の3つがある。

* HOBBIES
* HOUSEHOLD
* FOODS

カテゴリーの分析から以下のことがわかる。

* FOODSが商品数、販売数ともにもっとも多い
* 全体的に売り上げは増加している


In [ ]:
# カテゴリーの確認
sales['cat_id'].unique()

In [ ]:
# カテゴリーごとの商品数の集計
sales.groupby('cat_id').count()['id'].sort_values().plot(
    kind='barh',
    figsize=(15, 5),
    title='Count of Items by Category')
plt.show()

In [ ]:
# プロット
for i in sales['cat_id'].unique():
    items_col = [c for c in past_sales.columns if i in c]
    # 各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[items_col].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        alpha=0.8,
        title='Rolling 30 Day Average Total Sales by Category')
plt.legend(sales['cat_id'].unique())
plt.show()

# 2.4 店舗ごとの可視化
店舗は10店舗あり、CAに4店舗、TXとWIに3店舗ある。  
プロット結果から以下が考えられる。

* TXの店舗については、売り上げの店舗差はほとんどない
* WI_1, WI_2とCA_2は売り上げが急上昇する部分がある

In [ ]:
# 店舗の確認
sales['store_id'].unique()

In [ ]:
# 各店舗ごとの売り上げのリストの作成
store_list = prices['store_id'].unique()
store_list_ca = [s for s in store_list if 'CA' in s]
store_list_tx = [s for s in store_list if 'TX' in s]
store_list_wi = [s for s in store_list if 'WI' in s]

In [ ]:
# CAにおける各店舗ごとの売り上げのプロット
for s in store_list_ca:
    store_items = [c for c in past_sales.columns if s in c]
    # CAにおける各店舗について30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[store_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        ylim=[0,8000],
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (CA)')
plt.legend(store_list_ca)
plt.show()

In [ ]:
# TXにおける各店舗ごとの売り上げのプロット
for s in store_list_tx:
    store_items = [c for c in past_sales.columns if s in c]
    # TXにおける各店舗について30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[store_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        ylim=[0,8000],
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (TX)')
plt.legend(store_list_ca)
plt.show()

In [ ]:
# WIにおける各店舗ごとの売り上げのプロット
for s in store_list_wi:
    store_items = [c for c in past_sales.columns if s in c]
    # WIにおける各店舗について30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[store_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        ylim=[0,8000],
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (WI)')
plt.legend(store_list_ca)
plt.show()

# 2.5 部門ごとの可視化
部門はFOODSが3つ、HOBBIESとHOUSEHOLDが2つである。  
部門全体の売り上げのプロットと州別で見た部門の売り上げのプロットを行う。  
プロット結果から以下のことが考えられる。  

* 部門全体で見た時、各カテゴリーにおいてFOODS_3, HOBBIES_1, HOUSEHOLD_1が売り上げが多くなっている
* 上記の傾向は州別で見た時でも同様である


In [ ]:
# 部門の確認
sales['dept_id'].unique()

In [ ]:
# 各部門ごとの売り上げのリストの作成
d_list = sales['dept_id'].unique()
d_list_foods     = [d for d in d_list if 'FOODS' in d]
d_list_hobbies   = [d for d in d_list if 'HOBBIES' in d]
d_list_household = [d for d in d_list if 'HOUSEHOLD' in d]

In [ ]:
# FOODSの各カテゴリーにおける売り上げのプロット
for s in d_list_foods:
    department_items = [c for c in past_sales.columns if s in c]
    # FOODSの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[department_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (FOODS)')
plt.legend(d_list_foods)
plt.show()

In [ ]:
# HOBBIESの各カテゴリーにおける売り上げのプロット
for s in d_list_hobbies:
    department_items = [c for c in past_sales.columns if s in c]
    # HOBBIESの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[department_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (HOBBIES)')
plt.legend(d_list_hobbies)
plt.show()

In [ ]:
# HOUSEHOLDの各カテゴリーにおける売り上げのプロット
for s in d_list_household:
    department_items = [c for c in past_sales.columns if s in c]
    # HOUSEHOLDの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
    past_sales[department_items].sum(axis=1).rolling(30).mean().plot(
        figsize=(15, 5),
        alpha=0.8,
        color=next(color_cycle),
        title='Rolling 30 Day Average Total Sales (HOUSEHOLD)')
plt.legend(d_list_household)
plt.show()

In [ ]:
# FOODSにおいて、州別に見た各部門の売り上げのプロット
l = d_list_foods
for st in state_list:
    for d in l:
        store_items = [c for c in past_sales.columns if st in c]
        store_d_items = [s for s in store_items if d in s]
        # 州別のFOODSの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
        past_sales[store_d_items].sum(axis=1).rolling(30).mean().plot(
            figsize=(15, 5),
            ylim=[0,10000],
            alpha=0.8,
            title=f'Rolling 30 Day Average FOODS Sales ({st})')
    plt.legend(l)
    plt.show()

In [ ]:
# HOBBIESにおいて、州別に見た各部門の売り上げのプロット
l = d_list_hobbies
for st in state_list:
    for d in l:
        store_items = [c for c in past_sales.columns if st in c]
        store_d_items = [s for s in store_items if d in s]
        # 州別のHOBBIESの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
        past_sales[store_d_items].sum(axis=1).rolling(30).mean().plot(
            figsize=(15, 5),
            ylim=[0,2000],
            alpha=0.8,
            title=f'Rolling 30 Day Average HOBBIES Sales ({st})')
    plt.legend(l)
    plt.show()

In [ ]:
# HOUSEHOLDにおいて、州別に見た各部門の売り上げのプロット
l = d_list_household
for st in state_list:
    for d in l:
        store_items = [c for c in past_sales.columns if st in c]
        store_d_items = [s for s in store_items if d in s]
        # 州別のHOUSEHOLDの各カテゴリーについて30日分の売り上げの合計をとり平均を算出したものをプロット
        past_sales[store_d_items].sum(axis=1).rolling(30).mean().plot(
            figsize=(15, 5),
            ylim=[0,4000],
            alpha=0.8,
            title=f'Rolling 30 Day Average HOUSEHOLD Sales ({st})')
    plt.legend(l)
    plt.show()

# 2.6 まとめ
データの可視化からわかることを以下にまとめる。

* 時系列データとして扱うのが良さそう
* 日付(クリスマス)であるかどうかなどの特徴量の付与が必要になりそう
* 個別の商品ごとでそれぞれモデルを作れると良いかもしれない

In [ ]:
submit = pd.read_csv(osj('../input/submission/total-20201204-032508.csv'))

In [ ]:
submit.to_csv('submission.csv', index=False)